In [1]:
import numpy as np
import scipy.io as io
import glob
import h5py
import deepdish as dd
import dask.array as da
import os

### Read mat and write out as h5

In [2]:
matpath = '../../matdata/M3'
h5path = '../../H5data/M3'
fname = glob.glob(matpath+'/*huecircle*')

In [3]:
import re

for f in range(len(fname)):
    fname[f] = re.split('\.', re.split('/', fname[f])[-1])[0] 
fname

['Ro150126_s83a_huecircle_0003_nev',
 'Ro150127_s84a_huecircle_0003_nev',
 'Ro150128_s85a_huecircle_0003_nev',
 'Ro150129_s86a_huecircle_0003_nev',
 'Ro150210_s87a_huecircle_0003_nev',
 'Ro150211_s88a_huecircle_0003_nev',
 'Ro150212_289a_huecircle_0003_nev',
 'Ro150217_s92a_huecircle_0003_nev']

In [71]:
# Get the file pointer
thisfile = fname[8]
print matpath+'/'+thisfile+'.mat'
fid = h5py.File(matpath+'/'+thisfile+'.mat')
fid.keys()

IndexError: list index out of range

In [64]:
# Convert Eyes struct to dict
eyes_object = fid['Eyes']

trial_object = eyes_object['trial']
impath_object = eyes_object['impath']
imname_object = eyes_object['imname']

stim_onset_object = eyes_object['stim_onset']
stim_offset_object = eyes_object['stim_offset']

row_object = eyes_object['row']
col_object = eyes_object['col']

eyes = {}
for c in range(trial_object.size):
    # Get all the data for this struct element
    trial = int(np.array(fid[trial_object[c,0]])[0][0])
    impath = ''.join(chr(i) for i in fid[impath_object[c,0]])
    imname = ''.join(chr(i) for i in fid[imname_object[c,0]])
    
    stim_onset = np.array(fid[stim_onset_object[c,0]])[0][0]
    stim_offset = np.array(fid[stim_offset_object[c,0]])[0][0]
    
    row = np.array(fid[row_object[c,0]])[0][0]
    col = np.array(fid[col_object[c,0]])[0][0]
    
    # Create a dictionary for this element 
    this_dict = {}
    this_dict['trial'] = trial
    this_dict['impath'] = impath
    this_dict['imname'] = imname
    
    this_dict['stim_onset'] = stim_onset
    this_dict['stim_offset'] = stim_offset
    
    this_dict['row'] = row
    this_dict['col'] = col
    
    eyes[c] = this_dict
                         
print eyes[1]

{'stim_onset': 1.6805000000000001, 'imname': '000.jpg', 'trial': 1, 'col': 373.0, 'stim_offset': 1.9317333333333333, 'impath': 'stimuli/M3/Hues', 'row': 317.0}


In [65]:
# Convert Spikes struct to dict
spikes_object = fid['Spikes']
unit_object = spikes_object['unit']
times_object = spikes_object['times']

spikes = {}
for u in range(unit_object.size):
    uu = np.array(fid[unit_object[u,0]])
    tt = np.array(fid[times_object[u,0]])
    channel = int(np.floor(uu[0][0]))
    unit = int(10*(uu[0][0] - np.floor(uu[0][0])))
    unitid = '{0:0=2d}'.format(channel)+'{0:0=2d}'.format(unit)
    times = tt[0][:]
    spikes[unitid] = times

#for u in spikes:
#    print u
#    print spikes[u][:10]

spikes['5401'][:10]

array([ 1.38816667,  1.7056    ,  2.14953333,  2.163     ,  2.20666667,
        3.5913    ,  3.61083333,  4.37433333,  4.3794    ,  4.91773333])

In [66]:
# Convert features struct to dict
features_object = fid['features']
hue_object = features_object['hue']
features = {}
for c in range(hue_object.size):
    
    # Get all the data for this struct element
    hue = hue_object[0][c]
    
    # Create a dictionary for this element 
    this_dict = {}
    this_dict['hue'] = hue
    features[c] = this_dict
print features[0]

{'hue': 3.0475622895738499}


In [67]:
# Convert events struct to dict
# Convert features struct to dict
events_object = fid['events']
onset_object = events_object['onset']
offset_object = events_object['offset']
events = {}
for c in range(onset_object.size):
    
    # Get all the data for this struct element
    onset = onset_object[0][c]
    offset = offset_object[0][c]
    
    # Create a dictionary for this element 
    this_dict = {}
    this_dict['onset'] = onset
    this_dict['offset'] = offset
    
    events[c] = this_dict
print events[1]

{'onset': 1.6805000000000001, 'offset': 1.9317333333333333}


In [68]:
# Close the file
fid.close()

In [70]:
# Put everything into a bigger dict
d = {}
d['eyes'] = eyes
d['spikes'] = spikes
d['features'] = features
d['events'] = events

# Save it using deepdish
dd.io.save(h5path+'/'+thisfile+'.h5', d)

zlib


In [30]:
d['events']

{0: {'offset': 2.0444666666666667, 'onset': 1.8525},
 1: {'offset': 2.3618666666666668, 'onset': 2.1625666666666667},
 2: {'offset': 2.6636333333333333, 'onset': 2.4724333333333335},
 3: {'offset': 2.9743333333333335, 'onset': 2.7834333333333334},
 4: {'offset': 3.2863333333333333, 'onset': 3.0941999999999998},
 5: {'offset': 6.3319333333333336, 'onset': 6.1329000000000002},
 6: {'offset': 6.6345333333333336, 'onset': 6.4428000000000001},
 7: {'offset': 6.9436, 'onset': 6.7539666666666669},
 8: {'offset': 7.2436666666666669, 'onset': 7.0527666666666669},
 9: {'offset': 7.5667666666666671, 'onset': 7.3728666666666669},
 10: {'offset': 9.0027333333333335, 'onset': 8.8029666666666664},
 11: {'offset': 9.318366666666666, 'onset': 9.1229999999999993},
 12: {'offset': 9.6337333333333337, 'onset': 9.4429999999999996},
 13: {'offset': 9.9400666666666666, 'onset': 9.743033333333333},
 14: {'offset': 10.252633333333334, 'onset': 10.053000000000001},
 15: {'offset': 11.440066666666667, 'onset': 1